In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as colors
import numpy as np
from scipy import interpolate, stats

In [2]:
import ProcessOT2DOE as process

# Step 1: Load Sample Information
* Will be in the form of a dataframe. This dataframe should be standardized in the original documentation.

In [32]:
sample_info = pd.read_csv(r"C:\Users\Edwin\Desktop\02_26_21_OuzoScan\Sample_info") # make it find the blank position from sample_info
sample_info

,UID,dppc wtf,dspepeg200 wtf,pfh wtf,ethanol wtf,water wtf,Well,Labware,Slot
0,A1_2021-03-02,0.0001,0.000035,0.000,0.157895,0.841970,A1,Falcon 48 Well Plate 1500 ÂµL,2
1,A2_2021-03-02,0.0001,0.000035,0.000,0.210526,0.789339,A2,Falcon 48 Well Plate 1500 ÂµL,2
2,A3_2021-03-02,0.0001,0.000035,0.000,0.263158,0.736707,A3,Falcon 48 Well Plate 1500 ÂµL,2
3,A4_2021-03-02,0.0001,0.000035,0.000,0.315789,0.684076,A4,Falcon 48 Well Plate 1500 ÂµL,2
4,A5_2021-03-02,0.0001,0.000035,0.000,0.368421,0.631444,A5,Falcon 48 Well Plate 1500 ÂµL,2
...,...,...,...,...,...,...,...,...,...
232,F1_2021-03-02,0.0001,0.000035,0.001,0.736842,0.262023,F1,Falcon 48 Well Plate 1500 ÂµL,9
233,F2_2021-03-02,0.0001,0.000035,0.001,0.789474,0.209391,F2,Falcon 48 Well Plate 1500 ÂµL,9
234,F3_2021-03-02,0.0001,0.000035,0.001,0.842105,0.156760,F3,Falcon 48 Well Plate 1500 ÂµL,9
235,F4_2021-03-02,0.0001,0.000035,0.001,0.894737,0.104128,F4,Falcon 48 Well Plate 1500 ÂµL,9


# Step 2: Load UV-Vis Date
* ***If loading from plate reader***: Once the plate has been read, allow for exporting to an excel sheet and reformat for easy upload and merging.
    * The plate reader should automatically ask you if you would like to export to excel. When running multiple plates you can just keep selecting export and it will automatically add the new run as an additional sheet to the originally generated excel sheet. 
    * To format for easy loading:
        1. Create a new sheet in the same exported excel file and name it something simple (i.e. sheet1). 
        2. Copy the data from the sheet and add it to the newly created sheet. **NOTE**: Make sure to paste in the data as "only values". 
        3. Repeat for every single exported sheet (i.e. every plate)
* ***If loading from single cuvette reader***: 

* issue when over scanned on accident 

In [23]:
def merge_wavelength_dfs(df_list):
    merge_list = []
    for i, df in enumerate(df_list):
        if i == 0:
            df = df
        else: 
            df = df.drop(['Wavelength'])
        print(len(df))
        merge_list.append(df)
    return pd.concat(merge_list)

In [31]:
path = r"C:\Users\Edwin\Desktop\02_26_21_OuzoScan\02_26_21_OuzoScan\02_26_21_OuzoScan.xlsx"
plate_names = ['Sheet1','Sheet2','Sheet3','Sheet4','Sheet5']
plate_dfs = process.extract_plates(path, plate_names) # can edit/remove wells accidently measured etc, but really should be done at excel level
merged_df = merge_wavelength_dfs(plate_dfs) # a check
merged_df[148:]

49
48
48
48
48


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
A4,3.483,3.766,0.766,0.315,0.062,0.051,0.047,0.044,0.043,0.040,0.040,0.044,0.043,0.047,0.066
A5,3.467,3.745,0.682,0.273,0.058,0.048,0.046,0.043,0.041,0.040,0.041,0.047,0.046,0.049,0.069
A6,3.473,3.748,0.731,0.292,0.053,0.044,0.040,0.040,0.041,0.039,0.039,0.043,0.042,0.044,0.065
A7,3.484,3.761,0.793,0.321,0.053,0.044,0.041,0.040,0.039,0.037,0.037,0.041,0.040,0.043,0.064
A8,3.489,3.764,0.804,0.325,0.052,0.042,0.039,0.039,0.038,0.037,0.036,0.040,0.040,0.042,0.063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
F4,3.479,3.760,0.736,0.294,0.052,0.042,0.039,0.039,0.039,0.037,0.036,0.039,0.038,0.039,0.055
F5,3.478,3.755,0.736,0.291,0.053,0.044,0.041,0.041,0.040,0.038,0.037,0.039,0.039,0.039,0.052
F6,3.474,3.740,0.693,0.282,0.061,0.052,0.049,0.048,0.047,0.045,0.044,0.044,0.044,0.044,0.044
F7,3.485,3.751,0.750,0.306,0.062,0.052,0.048,0.048,0.047,0.045,0.044,0.044,0.043,0.043,0.044


# Step 2: Select/Locate Blank 
* ***If loading from plate reader***: Once the plate has been read, allow for exporting to an excel sheet and reformat for easy upload and merging.
    * The plate reader should automatically ask you if you would like to export to excel. When running multiple plates you can just keep selecting export and it will automatically add the new run as an additional sheet to the originally generated excel sheet. 
    * To format for easy loading:
        1. Create a new sheet in the same exported excel file and name it something simple (i.e. sheet1). 
        2. Copy the data from the sheet and add it to the newly created sheet. **NOTE**: Make sure to paste in the data as "only values". 
        3. Repeat for every single exported sheet (i.e. every plate)
* ***If loading from single cuvette reader***: 

In [11]:
merged_df

,200.0nm,250.0nm,300.0nm,350.0nm,400.0nm,450.0nm,500.0nm,550.0nm,600.0nm,650.0nm,700.0nm,750.0nm,800.0nm,850.0nm,900.0nm
0,200.000,250.000,300.000,350.000,400.000,450.000,500.000,550.000,600.000,650.000,700.000,750.000,800.000,850.000,900.000
1,3.501,3.764,0.778,0.322,0.052,0.045,0.042,0.042,0.042,0.041,0.041,0.049,0.046,0.053,0.065
2,3.490,3.770,0.755,0.315,0.054,0.044,0.040,0.040,0.040,0.038,0.038,0.045,0.043,0.049,0.063
3,3.477,3.747,0.681,0.283,0.058,0.048,0.043,0.042,0.042,0.039,0.039,0.045,0.043,0.049,0.064
4,3.473,3.741,0.659,0.273,0.060,0.049,0.045,0.043,0.042,0.039,0.038,0.045,0.043,0.048,0.065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,3.479,3.760,0.736,0.294,0.052,0.042,0.039,0.039,0.039,0.037,0.036,0.039,0.038,0.039,0.055
237,3.478,3.755,0.736,0.291,0.053,0.044,0.041,0.041,0.040,0.038,0.037,0.039,0.039,0.039,0.052
238,3.474,3.740,0.693,0.282,0.061,0.052,0.049,0.048,0.047,0.045,0.044,0.044,0.044,0.044,0.044
239,3.485,3.751,0.750,0.306,0.062,0.052,0.048,0.048,0.047,0.045,0.044,0.044,0.043,0.043,0.044


In [5]:
# baseline and combine
# baseline_series = merged_df.iloc[-1]
# merged_baselined_df = baseline_correction(merged_df, baseline_series)
combined_df = process.add_abs_to_sample_info(sample_info, merged_df)
combined_df

,UID,dppc wtf,dspepeg200 wtf,pfh wtf,ethanol wtf,water wtf,Well,Labware,Slot,200.0nm,...,450.0nm,500.0nm,550.0nm,600.0nm,650.0nm,700.0nm,750.0nm,800.0nm,850.0nm,900.0nm
0,A1_2021-03-02,0.0001,0.000035,0.000,0.157895,0.841970,A1,Falcon 48 Well Plate 1500 ÂµL,2.0,200.000,...,450.000,500.000,550.000,600.000,650.000,700.000,750.000,800.000,850.000,900.000
1,A2_2021-03-02,0.0001,0.000035,0.000,0.210526,0.789339,A2,Falcon 48 Well Plate 1500 ÂµL,2.0,3.501,...,0.045,0.042,0.042,0.042,0.041,0.041,0.049,0.046,0.053,0.065
2,A3_2021-03-02,0.0001,0.000035,0.000,0.263158,0.736707,A3,Falcon 48 Well Plate 1500 ÂµL,2.0,3.490,...,0.044,0.040,0.040,0.040,0.038,0.038,0.045,0.043,0.049,0.063
3,A4_2021-03-02,0.0001,0.000035,0.000,0.315789,0.684076,A4,Falcon 48 Well Plate 1500 ÂµL,2.0,3.477,...,0.048,0.043,0.042,0.042,0.039,0.039,0.045,0.043,0.049,0.064
4,A5_2021-03-02,0.0001,0.000035,0.000,0.368421,0.631444,A5,Falcon 48 Well Plate 1500 ÂµL,2.0,3.473,...,0.049,0.045,0.043,0.042,0.039,0.038,0.045,0.043,0.048,0.065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,F5_2021-03-02,0.0001,0.000035,0.001,0.947368,0.051497,F5,Falcon 48 Well Plate 1500 ÂµL,9.0,3.479,...,0.042,0.039,0.039,0.039,0.037,0.036,0.039,0.038,0.039,0.055
237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.478,...,0.044,0.041,0.041,0.040,0.038,0.037,0.039,0.039,0.039,0.052
238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.474,...,0.052,0.049,0.048,0.047,0.045,0.044,0.044,0.044,0.044,0.044
239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.485,...,0.052,0.048,0.048,0.047,0.045,0.044,0.044,0.043,0.043,0.044


In [33]:
combined_df.to_csv(r"C:\Users\Edwin\Desktop\OT2-DOE\02_26_21_OuzoScan\combined_data", index = False, header=True)

In [ ]:
test_open = pd.read_csv(r"C:\Users\Edwin\Desktop\11_18_2020\11_18_20_merged_info")
test_open

In [ ]:
# extract data by dict method of df calling series
wavelength = '400.0nm'
x_name = combined_df['Component 4 wtf'][0]
y_name = combined_df['Component 3 wtf'][0]
x = [float(i) for i in combined_df['Component 4 wtf'][1:].tolist()][:-1] #ethanol, 
y = [float(i) for i in combined_df['Component 3 wtf'][1:].tolist()][:-1] # pfh
z = [float(i) for i in combined_df[wavelength][1:].tolist()][:-1]
combined_restricted_xyz = [x,y,z]
modi = remove_visual_outliers(x,y,z,2) # this should only be used to find the new vmin and vmax but not to exclude